### 連線到 MySQL 資料庫

In [6]:
from sqlalchemy import create_engine, text

# 寫成 class 就能建立多個實例，連線到不同資料庫
class MySQL_DB:
    def __init__(self, user: str, password: str, db_name: str, host: str="localhost", port: int=3306) -> None:
        self.host = host
        self.port = str(port)
        self.user = user
        self.password = password # 通常是放在環境變數
        self.db_name = db_name
        self.engine = create_engine(f'mysql+pymysql://{self.user}:{self.password}@{self.host}:{self.port}')
    
    def create_db(self) -> None:
        with self.engine.connect() as connection:
            connection.execute(text(f"CREATE DATABASE IF NOT EXISTS {self.db_name} DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"))
            self.engine = create_engine(f'mysql+pymysql://{self.user}:{self.password}@{self.host}:{self.port}/{self.db_name}')
            print(f"資料庫 '{self.db_name}' 建立成功！")    

# 連線到 example_db 資料庫
db = MySQL_DB(user='root', password='example', db_name='example_db')
db.create_db()

資料庫 'example_db' 建立成功！


### 讀取 CSV 然後存入 MySQL

In [7]:
import pandas as pd
from os import path

# 讀取 interview.csv，使用 path.join 來支援不同作業系統
csv_file = path.join(".", "data", "interview.csv")

# 用 pandas 讀成 DataFrame
df = pd.read_csv(csv_file)

# 存入資料庫
df.to_sql('interview', con=db.engine, if_exists='replace', index=False)

# 印出資料庫裡的東西，確定有存進去
print(pd.read_sql('SELECT * FROM interview', con=db.engine))

         id name                 email  age                  city
0         1  蘇思穎     jie35@example.org   23  15594 屏東縣大安街6段131號4樓
1         2  萬俊傑    dliang@example.net   41        553 連江三民街48號之0
2         3  閻怡伶       eli@example.org   54     57672 臺中奇岩巷465號8樓
3         4  李佳樺    guyang@example.org   45        869 台北市民富巷5號之2
4         5  王嘉玲    weifan@example.net   37      15288 關山市光華街4號1樓
...     ...  ...                   ...  ...                   ...
9995   9996  尚惠婷      li60@example.com   32      68703 梅山縣自由巷2號之1
9996   9997  李志宏     leixu@example.net   25         365 中壢林森路4號4樓
9997   9998  邵志豪  duanyong@example.org   37    60090 鳳山縣長春街350號1樓
9998   9999  張怡伶    juan15@example.org   44     983 南投永安路4段469號之6
9999  10000  王詩婷    jungao@example.net   24      200 竹北龍山寺路7段8號之0

[10000 rows x 5 columns]
